In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    print(Model)
    purge(Model)
db.session.commit()

<class 'app.models.Bracket'>
<class 'app.models.Tournament'>
<class 'app.models.Match'>
<class 'app.models.Round'>
<class 'app.models.User'>


In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df

,id,role,username,email,password_hash,about_me,last_seen


In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df


,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$10ZyHB99$d470ccdf40a8d38e...,None,2020-03-23 21:55:30.435855


In [10]:
r.headers['location']

'http://127.0.0.1:5000/api/user/1'

In [11]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)
#     print(vars(r))

In [12]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$10ZyHB99$d470ccdf40a8d38e...,None,2020-03-23 21:55:30.435855
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$pMkB5z8I$3d8e723aa1fa3b84...,None,2020-03-23 21:55:30.636223
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$NcSJbZuN$822a324ed6b27844...,None,2020-03-23 21:55:30.728540
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$vJdOH8e8$469bd7d83ab4c5ff...,None,2020-03-23 21:55:30.808639
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$VRDO8bBt$af54345d4340062b...,None,2020-03-23 21:55:30.889122
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$3hEMPVPc$9f6d5a8f7defd877...,None,2020-03-23 21:55:30.968783
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$27OJ8FqI$6fcc2083300572f5...,None,2020-03-23 21:55:31.049110
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$ZqZ4zunr$9e2c72ecffdeb317...,None,2020-03-23 21:55:31.131070
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$F662ovg5$d6ed5bc47e639398...,None,2020-03-23 21:55:31.211187


In [13]:
q = """
select * from lobby_seeds
"""
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,user_id,lobby_id,seed


In [14]:
# whenever a user registers, the response contains the url to that user's homepage
r.headers['location']

'http://127.0.0.1:5000/api/user/9'

In [15]:
# login with miguel credentials
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/login

url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=('miguel', 'python'))

In [16]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [17]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMDUzMSwiZXhwIjoxNTg1MDAxMTMxfQ.eyJpZCI6MX0.jqla4XM1IMHK7yQLw97mKjUqK6pebQBK-k0KSAtL8gY8s7DuuFZEjcW9AerFBLxfbM-Reep0nxY6FVcCwQDK0g'

In [18]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [19]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourney"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content

b'{"Success":"Lobby created","lobby_id":1,"tournament_name":"miguel\'s tourney"}\n'

In [20]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourney,None,1


In [21]:
# now this user will create a tournament

In [22]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [23]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [24]:
# this our bracket (input to bracket creation):
tuple_list

[('Sunrisebanana', 5),
 ('Vik', 1),
 ('TPN', 6),
 ('Ptolemy', 4),
 ('Burnaby', 7),
 ('Spaceghost', 8),
 ('AngryFalco', 2),
 ('Kevin', 3)]

In [25]:
# create a json payload from the tuple list
url = 'http://127.0.0.1:5000/api/created-tournaments/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "users":usernames,
    "seeds":seeds,
    "tournament_name":"Fall Charity LAN 2018 Melee Singles"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r

<Response [200]>

In [26]:
q = \
"""
    select * from match where user_1 and user_2
"""

In [27]:
df = pd.read_sql_query(q, cnx)
df

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,d574a197-1f75-4f69-a604-e7f071ae6c59,6,None,8,None,None,1,5,10
1,2,3621c5f5-be10-4a3c-a0c8-9e40d9cb2db7,5,None,4,None,None,1,5,10
2,3,b17ddcee-90e2-4210-9cff-83431a38a431,3,None,9,None,None,1,6,11
3,4,367d41dd-c801-430b-ab8c-e6122964c041,7,None,2,None,None,1,6,11


In [28]:
active_match_ids = df.id.values
active_match_ids

array([1, 2, 3, 4])

In [29]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMDUzMiwiZXhwIjoxNTg1MDAxMTMyfQ.eyJpZCI6MX0._DjD8Gsz_rkyK2AuJWAg0SM59zkxQDLmVvLO-nOjbkJ5R65elx-VsLpl3lHCjrL0myw_ygeE_zHKPUb9RrUd1A'

In [30]:
r = requests.get(url, auth=('TPN', 'tpn'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMDUzMiwiZXhwIjoxNTg1MDAxMTMyfQ.eyJpZCI6Mn0.lLxuOjdWCE-RH5b-ziTpB2L1X_bb8dixHNo7A-Sry1TJsgiA5VwOnrp5bm4kDhBsevguu6ckeky-fqwe4s1FUw'

In [31]:
payload = json.dumps({
        "entrant1_score" : 3,
        "entrant2_score" : 2,
        "winner" : 9,
        "loser" : 8
})

In [32]:
url = f'http://127.0.0.1:5000/api/match/1/report-match'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content

b'{"Access Denied":"Only Tournament Organizers can report matches"}\n'

In [33]:
# for i in active_match_ids:
#     user_ids = [
#         df.loc[df['id'] == i].user_1.values[0],
#         df.loc[df['id'] == i].user_2.values[0]
#     ]
#     scores = random.sample(range(5), 2)
#     w_id = user_ids[np.argmax(scores)]
#     l_id = user_ids[np.argmin(scores)]
# #     print(w_id)
# #     print(l_id)
#     payload = json.dumps({
#         "entrant1_score" : str(scores[0]),
#         "entrant2_score" : str(scores[1]),
#         "winner" : str(w_id),
#         "loser" : str(l_id)
#     })
#     url = f'http://127.0.0.1:5000/api/match/{i}/report-match'
#     headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
#     r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
#     print(r.content)
#     break
    

b'{"Access Denied":"Only Tournament Organizers can report matches"}\n'


In [35]:
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "entrant1_score":4,
#     "entrant2_score":3,
#     "winner":,
#     "loser":,
# })

In [36]:

# TO = t.bracket.entrants[0]

# # initial post to db
# t.post_to_db(tournament_name, TO,)

NameError: name 't' is not defined

In [37]:
# # post self references in matches separately
# for r in t.bracket.rounds:
#     for m in r.matches:
#         m.post_self_refs()

In [38]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$10ZyHB99$d470ccdf40a8d38e...,None,2020-03-23 21:55:30.435855
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$pMkB5z8I$3d8e723aa1fa3b84...,None,2020-03-23 21:55:30.636223
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$NcSJbZuN$822a324ed6b27844...,None,2020-03-23 21:55:30.728540
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$vJdOH8e8$469bd7d83ab4c5ff...,None,2020-03-23 21:55:30.808639
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$VRDO8bBt$af54345d4340062b...,None,2020-03-23 21:55:30.889122
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$3hEMPVPc$9f6d5a8f7defd877...,None,2020-03-23 21:55:30.968783
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$27OJ8FqI$6fcc2083300572f5...,None,2020-03-23 21:55:31.049110
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$ZqZ4zunr$9e2c72ecffdeb317...,None,2020-03-23 21:55:31.131070
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$F662ovg5$d6ed5bc47e639398...,None,2020-03-23 21:55:31.211187


In [39]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id,finished
0,1,8,Fall Charity LAN 2018 Melee Singles,1,0


In [40]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [41]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,6,1
1,3,1
2,7,1
3,5,1
4,4,1
5,2,1
6,9,1
7,8,1


In [42]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [43]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,d574a197-1f75-4f69-a604-e7f071ae6c59,6.0,NaN,8.0,NaN,None,1,5.0,10.0
1,2,3621c5f5-be10-4a3c-a0c8-9e40d9cb2db7,5.0,NaN,4.0,NaN,None,1,5.0,10.0
2,3,b17ddcee-90e2-4210-9cff-83431a38a431,3.0,NaN,9.0,NaN,None,1,6.0,11.0
3,4,367d41dd-c801-430b-ab8c-e6122964c041,7.0,NaN,2.0,NaN,None,1,6.0,11.0
4,5,b692aa78-aa61-4419-ae77-92c6cc6d2975,NaN,NaN,NaN,NaN,None,2,7.0,12.0
5,6,317f29b6-263e-4598-a127-e393dfd6b6fd,NaN,NaN,NaN,NaN,None,2,7.0,13.0
6,7,731e3d15-1bdc-435a-99d0-833b5357d17f,NaN,NaN,NaN,NaN,None,3,8.0,15.0
7,8,6f3402bb-8149-4ca6-824a-4be8bf5138e4,NaN,NaN,NaN,NaN,None,4,9.0,9.0
8,9,15daa904-5154-482a-a2aa-a973e41b7925,NaN,NaN,NaN,NaN,None,5,NaN,NaN
9,10,bbfc1219-4fdd-4564-b0e9-06e03e5c443b,NaN,-1.0,NaN,0.0,None,6,12.0,NaN


In [44]:
url = 'http://127.0.0.1:5000/api/token'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({})
r = requests.get(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.dumps(r.content)

TypeError: Object of type 'bytes' is not JSON serializable